# Task1: Generate Dataset

In [1]:
import cv2

In [2]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
        
        # if face if not detected
        if faces is ():
            return None
        
        # if detected; we have to crop only face part; h = height, w = width
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    id = 1               # id of first authorized person
    img_id = 0            # no. of image of each authorized person
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            # (50,50) is the origin point from where text is to be written
            # font scale = 1, thickness = 2
            
            cv2.imshow("Cropped face", face)
        
        # 13 is the ASCII value of ENTER means if press enter infinite loop ends
        if cv2.waitKey(1)==13 or int(img_id)==200: 
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print("Samples Collected Successfully...")
    
generate_dataset()

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Ayush\AppData\Local\Temp\ipykernel_8084\4251263587.py:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Samples Collected Successfully...


# Task2: Train Classifier and Save it

In [3]:
import os
import cv2
from PIL import Image 
import numpy as np 

In [4]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        # convert image to gray scale image
        img = Image.open(image).convert('L')
        # change image to array format
        imageNp = np.array(img, 'uint8')
        # split the path of the image
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train the Classifier and save it
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    
train_classifier("data")

# Detect the Face and name it

In [5]:
import os
import cv2
from PIL import Image 
import numpy as np 

In [6]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
    coords = []
    
    for(x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)
        
        id, pred = clf.predict(gray_image[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
        
        if confidence>70:
            if id==1:
                cv2.putText(img, "Ayush", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==2:
                cv2.putText(img, "Siddhart", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
    
        coords=[x,y,w,h]
    return coords

def recognize(img, clf, faceCascade):
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255,255,255), "Face", clf)
    return img

# load classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow("face Detection", img)
    
    if cv2.waitKey(1)==13:
        break
        
video_capture.release()
cv2.destroyAllWindows()
